# Try different models-compare clustering results over eval. set
TODO: try different eps

In [3]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
#filename = 'pre_clustered_10_english.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)
print("#Number of clusters:",len(cdocs))
print("##Cluster length:",[str(len(cluster)) for cluster in cdocs])
print("### Doc length: ",[len(doc['title']+doc['abstract']) for cluster in cdocs for doc in cluster 
                          if not(doc['abstract'] is None)])
#print("### Doc length: ",[len(doc['headline']+doc['bodyText']) for cluster in cdocs for doc in cluster])

#Number of clusters: 10
##Cluster length: ['4', '3', '3', '2', '10', '9', '3', '4', '10', '6']
### Doc length:  [2210, 468, 2675, 1694, 1947, 1761, 1575, 1812, 1967, 2672, 2588, 2933, 950, 2218, 5673, 3585, 9133, 941, 1467, 9253, 2409, 2247, 3021, 1083, 1003, 1564, 2256, 1198, 2015, 8371, 1183, 3088, 1240, 1200, 1172, 1869, 2315, 3287, 1997, 2162, 2764, 996, 930, 877, 4216, 2068, 1992, 11060, 1157, 1350, 1225, 1978, 5780, 7297]


## Title+abstract models
## TODO: lower case text

In [2]:
# train different models
from gensim.utils import simple_preprocess as sp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]
# title + abstract models
train_corpus_ta = [ TaggedDocument(sp(doc['title'] + doc['abstract']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
#train_corpus_ta = [ TaggedDocument(sp(doc['headline'] + doc['bodyText']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
epochs = 45
vec_size = 100
models = [
    # dm = 0, simple SG, simpler model, most of the time efficient and accurate
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 100'),
    Doc2Vec(dm=0, vector_size=500, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 500, +Eps'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800'),
    Doc2Vec(dm=0, vector_size=800, negative=10, hs=0, min_count=2, sample=0, 
            epochs=40, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800, negative=10'),
    # dm = 1, CBOW equivalent 
    Doc2Vec(dm=1, vector_size= 100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=100, alpha=0.05'),
    Doc2Vec(dm=1, vector_size= 150, min_count=2,epochs = epochs, 
            workers=multiprocessing.cpu_count(), alpha= 0.5, comment='dm=1, vec=150, alpha=0.5, hs=1'),
    Doc2Vec(dm=1, vector_size= 500, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=500'),
    Doc2Vec(dm=1, vector_size= 500, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=500, alpha=.05')
]
# also evaluate result on currently used model
loaded_model = Doc2Vec.load('/home/nick/anaconda3/bin/Tirocinio/doc2vec_tryouts/Models_Live_Test/d2v_abstract&title0.model')
print("Model loaded")

# build our vocabulary of words (all the unique words encountered inside our corpus)
for model in models:
    model.build_vocab(train_corpus_ta)
    print(model, "-VocabSize:", len(model.wv.vocab))
print("Vocabulary created!")

# train the models on the given data!
counter = 0
for model in models:
    print("Training %s" % model)
    %time model.train(train_corpus_ta, total_examples=len(train_corpus_ta), epochs=model.epochs)
    #model.save(MODEL_NAME+str(counter)+'.model')
    counter = counter + 1
#print("Models Saved")

Model loaded
Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=0, vec = 500, +Eps",dbow,d500,n5,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=0, vec = 800, negative=10",dbow,d800,n10,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=1, vec=100, alpha=0.05",dm/m,d100,n5,w10,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=1, vec=150, alpha=0.5, hs=1",dm/m,d150,n5,w5,mc2,s0.001,t4) -VocabSize: 2104
Doc2Vec("dm=1, vec=500, lower eps, min_pts=3",dm/m,d500,n5,w10,mc2,t4) -VocabSize: 2104
Doc2Vec("dm=1, vec=500, alpha=.05",dm/m,d500,n5,w10,mc2,t4) -VocabSize: 2104
Vocabulary created!
Training Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
CPU times: user 1.62 s, sys: 0 ns, total: 1.62 s
Wall time: 907 ms
Training Doc2Vec("dm=0, vec = 500, +Eps",dbow,d500,n5,mc2,t4)
CPU times: user 3.24 s, sys: 16 ms, total: 3.25 s
Wall time: 1.7 s
Training Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
CPU times: user 5.5 s, sys: 40 ms, total: 5.54 s
Wa

# Evaluate Clustering

In [8]:
import utils
import my_dbscan
import model_evaluation as me
#import importlib
#importlib.reload(me)

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples
for k, model in enumerate(models):
    # try different eps for some model
    if k == 1:
        eps = 0.24
        eps_increment = 0.13
        min_s = 2
    elif k==6:
        eps = 0.11
        eps_increment = 0.13
        min_s = 3
    else:
        eps = 0.11
        eps_increment = 0.13
        min_s = 2
    # get list of document vectors
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment,
                                               n_iterations = 3, verbose = False, min_samples = min_s)
    # get clusters as list of titles
    titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
    clusters_found.append(titles_clusters)
    #data = utils.plot_clusters(titles_clusters)
    
    # evaluate clustering
    expected_clusters = []
    for docs in cdocs:
        expected_clusters.append([doc['title'] for doc in docs])
    precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters, alg='avg')
    print('Precision score: %s, Recall score: %s'%(precision, recall))
    
    models_precision.append(precision)
    models_recall.append(recall)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # that's way we have precision score
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len([l for l in urls_cluster_list if len(l)>0] ), len(cdocs)))

## loaded model
inferred_vecs = [loaded_model.infer_vector(sp(doc['title']+doc['abstract'])) for doc in unwrapped_docs]
urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = inferred_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment, n_iterations = 3, verbose = False)
titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
clusters_found.append(titles_clusters)
# evaluate clustering
expected_clusters = []
for docs in cdocs:
    expected_clusters.append([doc['title'] for doc in docs])
precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
print('Precision score: %s, Recall score: %s'%(precision, recall))
models_precision.append(precision)
models_recall.append(recall)
print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
    len(urls_cluster_list), len(cdocs)))

Accuracy (Precision) over each cluster:  [53.33333333333333, 75.0, 60.0, 100.0, 100.0, 83.33333333333333, 32.5, 66.66666666666666, 66.66666666666666, 50.0]
Accuracy (Recall) over each cluster:  [50.0, 100.0, 49.99999999999999, 100.0, 60.0, 25.925925925925924, 49.99999999999999, 50.0, 20.0, 33.33333333333332]
Precision score: 68.74999999999999, Recall score: 53.925925925925924
#Number of clusters found: 13, against number of pre-computed clusters: 10#

Accuracy (Precision) over each cluster:  [25.0, 25.0, 100.0, 100.0, 64.28571428571428, 100.0, 15.476190476190474, 33.33333333333333, 54.232804232804234, 47.22222222222222]
Accuracy (Recall) over each cluster:  [75.0, 100.0, 100.0, 100.0, 50.0, 100.0, 49.99999999999999, 100.0, 33.333333333333336, 41.66666666666666]
Precision score: 56.45502645502645, Recall score: 75.0
#Number of clusters found: 8, against number of pre-computed clusters: 10#

Accuracy (Precision) over each cluster:  [100.0, 75.0, 100.0, 100.0, 73.33333333333333, 87.5, 32.

In [12]:
for cluster in clusters_found[-2]:
    print(cluster)


['Tesla sotto indagine per colpa dei tweet di Elon Musk: crollo in Borsa - Corriere.it', 'Elon Musk denuciato per truffa, Tesla crolla in borsa - Wired']
["L'equinozio d'autunno non è il 21 settembre: quest'anno arriva il 23 - Repubblica.it", "E' l'equinozio d'autunno - Spazio & Astronomia - ANSA.it"]
['iPhone XS: perché Apple ha nascosto il notch?', 'Problemi per iPhone XS e XS Max: «Non si caricano se il cavo è collegato mentre lo schermo è spento» - Corriere.it']
['iPhone XS appiana le rughe, protestano gli utenti', 'iPhone XS: proteste per la carica troppo lenta']
['Fifa 19 contro Pes 2019: qual è il migliore quest’anno? Ecco la sfida giocata da noi - Corriere.it', 'Fifa 19, la prova in anteprima - Corriere.it']
['FIFA 19 sui campi della Champions League', 'FIFA 19, annunciata la disponibilità della demo', 'I miglioramenti di FIFA 19 per Nintendo Switch', "FIFA 19: L'ora dei campioni, trailer di lancio", "FIFA 19 si mostra nel trailer L'Ora dei Campioni"]
['A spasso attorno alla Lu

# Visualize results graphically

In [31]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
graph_name = 'model_eval_results_avg'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)


trace2 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall
data = [trace0, trace2]
layout = go.Layout(
    title = 'Clustering Accuracy Results (set-1)',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [5]:
from sklearn.metrics import silhouette_score
s_scores = []
titles = [doc['title'] for doc in unwrapped_docs]
urls = [doc['url'] for doc in unwrapped_docs]
    
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]

    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.11, eps_increment = 0.13, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
                break
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.17489207
0.18785714
0.21962713
0.14373659
0.16597174
-0.05085477
0.21195403
0.19973199


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.